# Quantum Phase Estimation

This notebook demonstrates the Qiskit `PhaseEstimation` component.

* [Qiskit HamiltonianGate documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.HamiltonianGate)
* [Qiskit PhaseEstimation documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.PhaseEstimation)

From the documentation:

In the Quantum Phase Estimation (QPE) algorithm, the Phase Estimation circuit is used to estimazte the phase $\phi$ of an eigenvalue $e^{2\pi i \phi}$ of a unitary operator $U$, provided with the corresponding eigenstate $|\psi\rangle$. That is
$$
U|\psi\rangle = e^{2\pi i \phi}|\psi\rangle.
$$
This estimation (eand thereby this circuit) is a central routine to several well-known algorithms, such as Shor's algorithm or Quantum Amplitude Estimation.

In [42]:
import math
import numpy as np
from IPython.display import display, Markdown
from math import log2
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import HamiltonianGate, PhaseEstimation, QFT, StatePreparation
from qiskit.quantum_info import Statevector
from qiskit.visualization.library import _generate_circuit_library_visualization

π = math.pi
# Pauli matrices: the operators
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Eigenvectors of Z
z0 = np.array([0, 1])
z1 = np.array([1, 0])
# Eigenvectors of Y
y0 = np.array([1, -1j]) / np.sqrt(2)
y1 = np.array([1, 1j]) / np.sqrt(2)

def make_circuit(op, e, apply_qpe=True):
    U = HamiltonianGate(op, -π/2, label='U')
    circuit = QuantumCircuit(3)
    circuit.append(StatePreparation(Statevector(e)), [2])
    if apply_qpe:
        qpe = PhaseEstimation(2, U)
        # _generate_circuit_library_visualization(qpe)
        circuit.append(qpe, [0, 1, 2])
    return circuit

def run_statevector(circuit):
    simulator = Aer.get_backend('statevector_simulator')
    aer_circuit = transpile(circuit, simulator)
    result_statevector = simulator.run(aer_circuit).result().get_statevector()
    return result_statevector

def escape_statevector(sv):
    return sv.draw('latex').data.replace('|', r'\|')

data = [
    {'opname': 'Z', 'ename': 'z_0', 'op': Z, 'e': z0},
    {'opname': 'Z', 'ename': 'z_1', 'op': Z, 'e': z1},
    {'opname': 'Y', 'ename': 'y_0', 'op': Y, 'e': y0},
    {'opname': 'Y', 'ename': 'y_1', 'op': Y, 'e': y1},
]

# make_circuit(Z, z0).draw('mpl')

table = "| Operator | Eigenvector | Input statevector | Output statevector |\n"
table += "| --- | --- | --- | --- |\n"
for d in data:
    opname = d['opname']
    ename = d['ename']
    input_statevector = escape_statevector(run_statevector(make_circuit(d['op'], d['e'], apply_qpe=False)))
    output_statevector = escape_statevector(run_statevector(make_circuit(d['op'], d['e'])))
    table += f"| ${opname}$ | ${ename}$ | {input_statevector} | {output_statevector} |\n"
display(Markdown(table))


| Operator | Eigenvector | Input statevector | Output statevector |
| --- | --- | --- | --- |
| $Z$ | $z_0$ | $$ \|100\rangle$$ | $$ \|111\rangle$$ |
| $Z$ | $z_1$ | $$ \|000\rangle$$ | $$ \|010\rangle$$ |
| $Y$ | $y_0$ | $$\frac{\sqrt{2}}{2} \|000\rangle- \frac{\sqrt{2} i}{2} \|100\rangle$$ | $$\frac{\sqrt{2}}{2} \|011\rangle- \frac{\sqrt{2} i}{2} \|111\rangle$$ |
| $Y$ | $y_1$ | $$\frac{\sqrt{2}}{2} \|000\rangle+\frac{\sqrt{2} i}{2} \|100\rangle$$ | $$\frac{\sqrt{2}}{2} \|010\rangle+\frac{\sqrt{2} i}{2} \|110\rangle$$ |


In [53]:
input_qubit = 0
n = 2
qubits = QuantumRegister(n+1, "q")
clbits = ClassicalRegister(n, "c")
circuit = QuantumCircuit(qubits, clbits)
circuit.x(input_qubit) # for eigenvector |1>
for i in range(n):
    circuit.h(i+1)
for i in range(n):
    if n-i == 1:
        circuit.cz(n-i, input_qubit)
circuit.append(QFT(n).inverse(), qubits[1:])
fig = circuit.draw('mpl')
sv = run_statevector(circuit)
display(Markdown(escape_statevector(sv)))


$$ \|101\rangle$$

In [57]:
input_qubit = 0
n = 2
qubits = QuantumRegister(n+1, "q")
clbits = ClassicalRegister(n, "c")
circuit = QuantumCircuit(qubits, clbits)
# circuit.rx(math.pi/2, input_qubit)
circuit.rx(-math.pi/2, input_qubit)
for i in range(n):
    circuit.h(i+1)
for i in range(n):
    if n-i == 1:
        circuit.cy(n-i, input_qubit)
circuit.append(QFT(n).inverse(), qubits[1:])
fig = circuit.draw('mpl')
sv = run_statevector(circuit)
display(Markdown(escape_statevector(sv)))

$$\frac{\sqrt{2}}{2} \|000\rangle+\frac{\sqrt{2} i}{2} \|001\rangle$$

In [ ]:
data = [
    {'operator': 'Z', 'eigenvector': z0},
    {'operator': 'Z', 'eigenvector': z1},
    {'operator': 'Y', 'eigenvector': y0},
    {'operator': 'Y', 'eigenvector': y1},
]

def run_phase_estimation(operator, eigenvector, apply_qpe=True):
    n = 2
    qubits = QuantumRegister(n+1, "q")
    clbits = ClassicalRegister(n, "c")
    circuit = QuantumCircuit(qubits, clbits)
    circuit.append(StatePreparation(eigenvector), [0])
    if apply_qpe:
        for i in range(n):
            circuit.h(i+1)
        for i in range(n):
            if operator == 'Z':
                if n-i == 1:
                    circuit.cz(n-i, input_qubit)
            elif operator == 'Y':
                if n-i == 1:
                    circuit.cy(n-i, 0)
            else:
                raise ValueError(f"Unknown operator: {operator}")
        circuit.append(QFT(n).inverse(), qubits[1:])
    return run_statevector(circuit)

table = "| Operator | Input statevector | Output statevector |\n"
table += "| --- | --- | --- |\n"
for d in data:
    operator = d['operator']
    eigenvector = d['eigenvector']
    input_statevector = escape_statevector(run_phase_estimation(operator, eigenvector, apply_qpe=False))
    output_statevector = escape_statevector(run_phase_estimation(operator, eigenvector))
    table += f"| ${operator}$ | {input_statevector} | {output_statevector} |\n"
display(Markdown(table))